In [ ]:
# 1 加载必要的库

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
# 2 定义超参数

BATCH_SIZE = 16 # 每批处理的数据
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 是否用GPU
EPOCHS = 10 # 训练数据的轮次

In [ ]:
DEVICE

device(type='cpu')

In [ ]:
# 3 构建pipeline，对图形做处理

pipeline = transforms.Compose([
    transforms.ToTensor(), # 将图片转换成tensor
    transforms.Normalize((0.1307,), (0.3081,)) # 归一化：降低模型复杂度
])

In [ ]:
# 4 下载、加载数据集

from torch.utils.data import DataLoader

# 下载数据集

train_set = datasets.MNIST("data", train=True, download=True, transform=pipeline)
test_set = datasets.MNIST("data", train=False, download=True, transform=pipeline)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 15081958.57it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 469663.83it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4390643.51it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3589022.00it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [ ]:
# 加载数据集

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)

test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# 5 构建网络模型

class Digit(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,10,5) # 1: 灰度图片的通道，10：输出通道，5：kernel
        self.conv2 = nn.Conv2d(10,20,3) # 10: 输入通道，20: 输出通道，3: kernel
        self.fc1 = nn.Linear(20*10*10, 500) # 20*10*10: 输入通道， 500: 输出通道
        self.fc2 = nn.Linear(500, 10) # 500: 输入通道，10:输出通道

    def forward(self, x):
        input_size = x.size(0) # batch_size
        x = self.conv1(x) # 输入: batch*1*28*28, 输出: batch*10*24*24 （28-5+1）
        x = F.relu(x) # 保持shape不变，输出：batch*10*24*24
        x = F.max_pool2d(x, 2, 2) # 输入：batch*10*24*24, 输出: batch*10*12*12
        x = self.conv2(x) # 输入: batch*10*12*12, 输出: batch*20*10*10
        x = F.relu(x) # 保持shape不变，输出：batch*20*10*10
        x = x.view(input_size, -1) # 拉平，-1 自动计算维度， 20*10*10=2000
        x = self.fc1(x) # 输入: batch*2000, 输出: batch*500
        x = F.relu(x) # 保持shape不变，输出: batch*500
        x = self.fc2(x) # 输入: batch*500, 输出: 10
        output = F.log_softmax(x, dim=1) # 计算分类后，每个数字的概率值

        return output

In [ ]:
# 6 定义优化器

model = Digit().to(DEVICE)

optimizer = optim.Adam(model.parameters())

In [ ]:
# 7 定义训练方法

def train_model(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_index, (data, target) in enumerate(train_loader):
        # 部署到device上
        data, target = data.to(device), target.to(device)
        # 梯度初始化为0
        optimizer.zero_grad()
        # 训练后的结果
        output = model(data)
        # 计算损失
        loss = F.cross_entropy(output, target)
        # 反向传播
        loss.backward()
        # 参数优化
        optimizer.step()
        if batch_index % 3000 == 0:
            print("Train Epoch : {} \t Loss : {:.6f}".format(epoch, loss.item()))

In [ ]:
# 8 定义测试方法

def test_model(model, device, test_loader):
    # 模型验证
    model.eval()
    # 正确率
    correct = 0.0
    # 测试损失
    test_loss = 0.0
    # 不会进行计算梯度，也不会进行反向传播
    with torch.no_grad():
        for data, target in test_loader:
            # 部署到device上
            data, target = data.to(device), target.to(device)
            # 测试数据
            output = model(data)
            # 计算损失
            test_loss += F.cross_entropy(output, target).item()
            # 找到概率值最大的下标
            pred = output.max(1, keepdim=True)[1]
            #pred = torch.max(output, dim=1)
            # pred = output.argmax(dim=1)
            # 累计正确的值
            correct += pred.eq(target.view_as(pred)).sum().item()
        # 计算平均loss
        test_loss /= len(test_loader.dataset)
        print("Test -- Average Loss : {:.4f}, Accuracy : {:.3f}\n".format(test_loss, correct/len(test_loader.dataset)*100.0))

In [ ]:
# 9 调用方法(7和8)

for epoch in range(1, EPOCHS+1):
    train_model(model, DEVICE, train_loader, optimizer, epoch)
    test_model(model, DEVICE, test_loader)

Train Epoch : 1 	 Loss : 2.295543
Train Epoch : 1 	 Loss : 0.062351
Test -- Average Loss : 0.0031, Accuracy : 98.490

Train Epoch : 2 	 Loss : 0.002531
Train Epoch : 2 	 Loss : 0.000348
Test -- Average Loss : 0.0027, Accuracy : 98.690

Train Epoch : 3 	 Loss : 0.001374
Train Epoch : 3 	 Loss : 0.024123
Test -- Average Loss : 0.0027, Accuracy : 98.810

Train Epoch : 4 	 Loss : 0.008191
Train Epoch : 4 	 Loss : 0.011870
Test -- Average Loss : 0.0023, Accuracy : 98.990

Train Epoch : 5 	 Loss : 0.001290
Train Epoch : 5 	 Loss : 0.000185
Test -- Average Loss : 0.0032, Accuracy : 98.840

Train Epoch : 6 	 Loss : 0.189695
Train Epoch : 6 	 Loss : 0.199310
Test -- Average Loss : 0.0045, Accuracy : 98.350

Train Epoch : 7 	 Loss : 0.000135
Train Epoch : 7 	 Loss : 0.000012
Test -- Average Loss : 0.0030, Accuracy : 98.970

Train Epoch : 8 	 Loss : 0.000000
Train Epoch : 8 	 Loss : 0.000002
Test -- Average Loss : 0.0030, Accuracy : 99.000

Train Epoch : 9 	 Loss : 0.000027
Train Epoch : 9 	 Loss